In [1]:
import requests
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
token="здесь должен быть токен"

In [3]:
group_name="expert_news"
offset=0
count=100
all_posts=[]
all_comments=[]

In [4]:
offset=0
while offset<2000:
        req=requests.get('https://api.vk.com/method/wall.search',
               params={
                   'access_token': token,
                   'v': 5.131,
                   'domain': group_name,
                   'owners_only': 1,
                   'query': 'санкци',
                   'count': count,
                   'offset': offset
                   
               })
        res=req.json()['response']['items']
        offset+=100
        time.sleep(1)
        all_posts.extend(res)

In [5]:
res=req.json()['response']['items']
all_posts.extend(res)
df = pd.json_normalize(all_posts)

In [6]:
df=df[["id", "owner_id", "date", "text", "comments.count"]]
df.rename(columns={'date': 'post_time','text':'post_text'}, inplace=True)
df=df[(df['post_time']>=1630454400) & (df['post_time']<1675209600)]
df

,id,owner_id,post_time,post_text,comments.count
9,261222,-26517977,1674718200,"Экономические потрясения, вызванные санкционно...",0
10,261187,-26517977,1674585327,«НоваТЭК» как будто не замечает тотальных санк...,0
11,261165,-26517977,1674536522,Российский рынок акций продолжил подъем на пуб...,0
12,261137,-26517977,1674408530,Промышленность минеральных удобрений оказалась...,0
13,260963,-26517977,1673619615,Если в ближайшее время выйти на такие объемы н...,8
...,...,...,...,...,...
350,245703,-26517977,1632495900,Палата представителей США приняла оборонный бю...,0
351,245514,-26517977,1632073846,Российские рынки переживают период восстановле...,1
352,244986,-26517977,1630755073,В Высшей школе экономики оценили эффект россий...,11
353,244937,-26517977,1630648860,Холдинговая компания Sovco Capital Partners S....,1


In [7]:
owner_id=-26517977
posts_id=list(df[df['comments.count']!=0]['id'])

In [8]:
for post_id in posts_id:
    req=requests.get('https://api.vk.com/method/wall.getComments',
               params={
                   'access_token': token,
                   'v': 5.131,
                   'owner_id': owner_id,
                   'post_id': post_id,
                   'count': 100
               })
    time.sleep(1)
    res=req.json()['response']['items']
    all_comments.extend(res)

In [9]:
data = pd.json_normalize(all_comments)
data

,id,from_id,date,text,post_id,owner_id,parents_stack,thread.count,thread.items,thread.can_post,thread.show_reply_button,attachments,deleted
0,260964,51377480,1673622861,"Призыв, верный! Но похож на мечту. Для такого ...",260963.0,-26517977.0,[],0,[],True,True,NaN,NaN
1,260965,94373551,1673623917,Всё никак и никто не сможет импортозаместить. ...,260963.0,-26517977.0,[],0,[],True,True,NaN,NaN
2,260968,12697056,1673629559,"Систему образования надо настраивать так, чтоб...",260963.0,-26517977.0,[],0,[],True,True,NaN,NaN
3,260972,77342254,1673636428,Нужны заинтересованность и настойчивость власт...,260963.0,-26517977.0,[],0,[],True,True,NaN,NaN
4,260973,557208055,1673645613,"Если пройтись по нашим техническим ВУЗам, то м...",260963.0,-26517977.0,[],3,[],True,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,244999,631538347,1630773676,"Ничего подобного. Когда свой огород, еды много...",244986.0,-26517977.0,[],1,[],True,True,NaN,NaN
590,245006,786521,1630782100,Пора отписаться..,244986.0,-26517977.0,[],2,[],True,True,NaN,NaN
591,245034,786521,1630851344,"""..дура необучаемая с большим чсв с возу кобыл...",244986.0,-26517977.0,[],0,[],True,True,NaN,NaN
592,244972,308200436,1630714071,Расшифровка: бесполезная помойка переезжает в РФ.,244937.0,-26517977.0,[],0,[],True,True,NaN,NaN


In [10]:
data=data[["post_id", "text", "date", "from_id"]]
data.rename(columns={'date': 'comment_time', 'id': 'comment_id'}, inplace=True)
data=data.dropna()
data['post_id'] = data['post_id'].astype(int)
data

,post_id,text,comment_time,from_id
0,260963,"Призыв, верный! Но похож на мечту. Для такого ...",1673622861,51377480
1,260963,Всё никак и никто не сможет импортозаместить. ...,1673623917,94373551
2,260963,"Систему образования надо настраивать так, чтоб...",1673629559,12697056
3,260963,Нужны заинтересованность и настойчивость власт...,1673636428,77342254
4,260963,"Если пройтись по нашим техническим ВУЗам, то м...",1673645613,557208055
...,...,...,...,...
589,244986,"Ничего подобного. Когда свой огород, еды много...",1630773676,631538347
590,244986,Пора отписаться..,1630782100,786521
591,244986,"""..дура необучаемая с большим чсв с возу кобыл...",1630851344,786521
592,244937,Расшифровка: бесполезная помойка переезжает в РФ.,1630714071,308200436


In [11]:
df.to_csv('expert_posts_s.csv', index=False)

In [12]:
data.to_csv('expert_comments_s.csv', index=False)